## Problem 2

This is the code I used for my computation of the probability. get_prob() is a recursive function and seq_prob() is a wrapper function that makes recursive calls after calculating initializing the probability. More information about the implementation is in my write-up. 

In [2]:
states = [[0.8, 0.2], [0.05, 0.95]]
bases = { 'A': [0.2, 0.1],
          'C': [0.5, 0.25],
          'G': [0.1, 0.25],
          'T': [0.2, 0.4]
        }
        
p_actg = 0.0

def seq_prob(seq): 
    # initial probability is 0.5
    return 0.5 * bases[seq[0:1]][0] * get_prob(0, 0, seq[1:]) + \
           0.5 * bases[seq[0:1]][0] * get_prob(0, 1, seq[1:]) + \
           0.5 * bases[seq[0:1]][1] * get_prob(1, 1, seq[1:]) + \
           0.5 * bases[seq[0:1]][1] * get_prob(1, 0, seq[1:])
           
def get_prob(prevstate, currstate, seq): 
    # base case: one base left
    if len(seq) == 1:
        # return probability of last base
        return states[prevstate][currstate] * bases[seq][currstate]
    else: 
        curr_base = seq[0:1]
        # obtain the probabilities and multiply by recursive call on the smaller case (shave down base)
        return (states[prevstate][currstate] * bases[curr_base][currstate]) * (get_prob(currstate, 0, seq[1:]) + get_prob(currstate, 1, seq[1:]))
        
print(seq_prob("ACGT"))


0.0031796718750000006


## Problem 3

I wrote some code to output the sequences as defined in the problem. prob_help() is a recursive function and probability_tree() is the wrapper function. The recursive function works by recursively traversing a HMM tree. 

In [9]:
import numpy as np
from itertools import combinations

# prints out expression of the probability given a sequence and labels
def probability_tree(labels, sequence):
    terms = []
    # initialize with the initial probability (init())
    for l in labels: 
        tmp = []
        tmp.append("init({})*{}_{}".format(l, l, sequence[0]))
        terms.append(tmp.copy())
    # branch out for recursion
    for i in range(len(labels)): 
        for j in range(len(labels)):
            prob_help(labels, terms[i], labels[i], labels[j], sequence[1:], '')
    # basic string parsing 
    retstr = ''
    for t in terms: 
        retstr += "+ {}*(".format(t[0])
        for l in t[1:len(t)-1]: 
            retstr += l + " + "
        retstr += t[len(t)-1] + ") "
    # prints out human friendly math
    print(retstr[2:]+"\n")
    # returns a list of terms split into [['init(A)*...'], ['init(B)*...']]
    # for later use
    return terms

def prob_help(labels, terms, prev_state, curr_state, seq, prob): 
    # last emission reached
    if len(seq) == 1:
        prob += "*{}{}*{}_{}".format(prev_state, curr_state, curr_state, seq)
        # add the probability built up to this point to a list
        terms.append(prob[1:])
    else:
        prob += "*{}{}*{}_{}".format(prev_state, curr_state, curr_state, seq[0])
        # branch out into all possible states
        for i in range(len(labels)): 
            # continue to build the probability until the last emission is reached
            # shave down the sequence by one emission
            prob_help(labels, terms, curr_state, labels[i], seq[1:], prob)
        
print("P_111")
t111 = probability_tree(["A", "B"], "111")
print("P_000")
t000 = probability_tree(["A", "B"], "000")
print("P_110")
t110 = probability_tree(["A", "B"], "110")
print("P_011")
t011 = probability_tree(["A", "B"], "011")
print("P_100")
t100 = probability_tree(["A", "B"], "100")
print("P_001")
t001 = probability_tree(["A", "B"], "001")

P_111
init(A)*A_1*(AA*A_1*AA*A_1 + AA*A_1*AB*B_1 + AB*B_1*BA*A_1 + AB*B_1*BB*B_1) + init(B)*B_1*(BA*A_1*AA*A_1 + BA*A_1*AB*B_1 + BB*B_1*BA*A_1 + BB*B_1*BB*B_1) 

P_000
init(A)*A_0*(AA*A_0*AA*A_0 + AA*A_0*AB*B_0 + AB*B_0*BA*A_0 + AB*B_0*BB*B_0) + init(B)*B_0*(BA*A_0*AA*A_0 + BA*A_0*AB*B_0 + BB*B_0*BA*A_0 + BB*B_0*BB*B_0) 

P_110
init(A)*A_1*(AA*A_1*AA*A_0 + AA*A_1*AB*B_0 + AB*B_1*BA*A_0 + AB*B_1*BB*B_0) + init(B)*B_1*(BA*A_1*AA*A_0 + BA*A_1*AB*B_0 + BB*B_1*BA*A_0 + BB*B_1*BB*B_0) 

P_011
init(A)*A_0*(AA*A_1*AA*A_1 + AA*A_1*AB*B_1 + AB*B_1*BA*A_1 + AB*B_1*BB*B_1) + init(B)*B_0*(BA*A_1*AA*A_1 + BA*A_1*AB*B_1 + BB*B_1*BA*A_1 + BB*B_1*BB*B_1) 

P_100
init(A)*A_1*(AA*A_0*AA*A_0 + AA*A_0*AB*B_0 + AB*B_0*BA*A_0 + AB*B_0*BB*B_0) + init(B)*B_1*(BA*A_0*AA*A_0 + BA*A_0*AB*B_0 + BB*B_0*BA*A_0 + BB*B_0*BB*B_0) 

P_001
init(A)*A_0*(AA*A_0*AA*A_1 + AA*A_0*AB*B_1 + AB*B_0*BA*A_1 + AB*B_0*BB*B_1) + init(B)*B_0*(BA*A_0*AA*A_1 + BA*A_0*AB*B_1 + BB*B_0*BA*A_1 + BB*B_0*BB*B_1) 



**After** obtaining these terms, I wrote some loops and parsing to obtain the possibilities that P_111 and P_000 are both equal to 0, in the case that init(A) and init(B) are positive. 

### Case 3: initial probabilities are both non-zero 

In [8]:
# All possible terms
p = ["AA", "AB", "BA", "BB", "A_0", "A_1", "B_0", "B_1"]
# All combinations max length 3
# if there are more than 3 terms that need to become 0 for the equation to be true, then
# the matrices derived are already invalid (either 3 zeroes in one matrix or 2 zeroes in
# each matrix)
comb = [list(combinations(p, i)) for i in range(4)]
# Combinations that make P000 = 0
zero_combs000 = []
# ... that make P111 = 0
zero_combs111 = []
# iterate through combinations
for c in comb: 
    for i in c: 
        # loop through p000 
        tmp = [j for i in t000 for j in i]
        # removes the init term
        tmp = [e for e in tmp if "init" not in e]
        for t in range(len(tmp)): 
            # if the term is in the combination and the term, set it to 0
            if tmp[t] != 0 and any(s in tmp[t] for s in i): 
                tmp[t] = 0
            # see if the sum of terms is 0
            if np.array_equal(np.asarray(tmp), np.zeros((1, len(tmp)))[0]): 
                zero_combs000.append(i)
        # loop through p111
        tmp = [j for i in t111 for j in i]
        # removes the init term
        tmp = [e for e in tmp if "init" not in e]
        for t in range(len(tmp)): 
            # if the term is in the combination and the term, set it to 0
            if tmp[t] != 0 and any(s in tmp[t] for s in i): 
                tmp[t] = 0
            # see if the sum of terms is 0
            if np.array_equal(np.asarray(tmp), np.zeros((1, len(tmp)))[0]): 
                zero_combs111.append(i)
        
# obtains the intersection of combinations where P_111 = 0 and P_000 = 0.
print([i for i in list(set(zero_combs000) & set(zero_combs111))])

[('AA', 'AB', 'BB'), ('AA', 'B_0', 'B_1'), ('BB', 'A_0', 'A_1'), ('AA', 'BA', 'BB')]


As described in my write-up, none of these combinations are valid transition or emission matrices. Thus, there are no good values of substitution to make P_000 = P_111 = 0 when both initial probabilities are non-zero. 

## Problem 4: Viterbi's Algorithm

This is my implementation of Viterbi's. 

In [5]:
def viterbi(seq, transition, emission, initial, labels):
    # converts sequence to an array
    seq = np.array([int(i) for i in seq])
    # number of distinct states
    n_states = len(labels)
    # length of sequence (n value)
    N = len(seq)
    # Matrix that contains the max probability of reaching each state
    max_probs = np.zeros((n_states, N + 1))
    max_probs[:,0] += initial * emission[:, seq[0]]
    # max_probs[initial, 0] = 1
    # Matrix that contains the previous state which corresponds 
    # to the maximum probability of reaching the current state
    max_state_combs = np.zeros((n_states, N + 1), dtype='int')
    for i in range(1, N + 1):
        for j in range(n_states):
            # obtain probability from the maximal jump (transition)
            max_probs[j, i] = np.max(max_probs[:, i - 1] * transition[:, j])
            # Store this jump for later reference
            max_state_combs[j, i] = np.argmax(max_probs[:, i - 1] * transition[:, j])
            # multiply with probability of current state (emission)
            max_probs[j, i] *= emission[j, seq[i - 1]]
    # obtain the maximal state sequence
    max_states = np.zeros(N + 1, dtype='int')
    max_states[N] = np.argmax(max_probs[:, N])
    # tracing backwards for the desired sequence 
    for n in range(N, 1, -1):
        max_states[n - 1] = max_state_combs[max_states[n], n - 1]
    retstr = ''
    for s in max_states[1:]: 
        retstr += labels[s]
    return retstr

The function above is coded according to what was taught by Professor Pachter in class. There is an initial probability of 0.5, so the first column is initialized as such. Then, there are nested for loops which go through the matrix of n_states x len(sequence). The max probability at each cell is kept track of, and a separate matrix is used to keep track of where the jumps of max probability originated from, so that we can backtrack later and find the maximal state sequence. 

In [6]:
transition = np.array([
    [0.1, 0.9],
    [0.1, 0.9]
])
emission = np.array([
    [0.1, 0.9],
    [0.1, 0.9]
])

seq = "0100010010111100111111111000000000011001000010111111111110011111111111110111111010010100001000000000"
initial = [0.5, 0.5]
names = ['A', 'B']
print(viterbi(seq, transition, emission, initial, names))

ABBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB


I used two forms of verification for my code. First, the transition and emission matrix are written in a way that the outputted state sequence makes sense, since the probability of B -> B is very high. Additionally, I verified this sequence using the HMM class provided, which also outputted the state sequence shown above when given the transition and emission matrices, along with the 100 base sequence and the state names. 